In [ ]:
import requests

def fetch_air_range(station_id, date_from, date_until):
    # Define the URL of the endpoint that provides the air quality data
    url = 'http://airviro.klab.ee/station/csv'
    
    # Define the parameters to be sent with the POST request to the endpoint
    data = {
        'filter[type]': 'INDICATOR',
        'filter[cancelSearch]': '',
        'filter[stationId]': station_id,
        'filter[dateFrom]': date_from,
        'filter[dateUntil]': date_until,
        'filter[submitHit]': '1',
        'filter[indicatorIds]': ''
    }
    
    # Send a POST request to the endpoint with the specified parameters
    response = requests.post(url, data)
    
    # Return the text content of the response
    return response.text

import datetime

def get_first_and_last_day_of_month(year, month):
    # Get the first day of the month
    first_day = datetime.date(year, month, 1)

    # Get the number of days in the month
    if month == 12:
        num_days = 31
    else:
        num_days = (datetime.date(year, month+1, 1) - datetime.timedelta(days=1)).day

    # Get the last day of the month
    last_day = datetime.date(year, month, num_days)

    return first_day, last_day


Apply functions from the last cell to iterate over the moths of year 2022, fetch air quality infromation for each month and write it into a file.

In [ ]:
import os
# Define the year for which to fetch air quality data
year = 2022

os.makedirs('air_data', exist_ok=True)

# Loop through the months of the year and fetch air quality data for each month
for month in range(1, 13):
    # Get the first and last day of the month
    first_day, last_day = get_first_and_last_day_of_month(year, month)

    # Print the first and last day of the month for debugging purposes
    print(first_day, last_day)

    # Fetch the air quality data for the month
    data = fetch_air_range(8, first_day, last_day)

    # Write the data to a CSV file
    with open(f'air_data/air_{year}_{month}.csv', 'w') as f:
        f.write(data)

It would be cumbersome to check each individual month file. Stich them together into single year file.

In [ ]:
# Define the year for which to consolidate air quality data
year = 2022

# Define an empty list to store the data
data = []

# Loop through the months of the year and append the data to the list
for month in range(1, 13):
    # Open the CSV file for the month
    with open(f'air_data/air_{year}_{month}.csv', 'r') as f:
        # If this is the first month, store the header row
        if month == 1:
            header = f.readline()
            data.append(header)
        # Otherwise, skip the header row
        else:
            f.readline()
        # Append the remaining data to the list
        data.extend(f.readlines())

# Write the consolidated data to a CSV file
with open(f'air_data/air_{year}.csv', 'w') as f:
    f.writelines(data)


After the data has been checked and converted using `OpenRefine` software using the steps from `data_tranform_steps_air.json`, we can create sqlite3 database file uing following commands:
```
# in terminal (and in air_data folder)
sqlite3 air-2022.db
# on sqlite3 prompt
.read air-2022.sql
.exit
```

After that we can open the `sqlite3` database and connect to it using package of the same name (needs to be installed).
Using `pandas` package we can easily calculate day and month average metrics for air data and create corresponding new
tables to the given database file.

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('air_data/air-2022.db')

# Select the data from the database
df = pd.read_sql_query("SELECT * FROM air_2022", conn)

# Convert the date column to a datetime data type
df['DateTime'] = pd.to_datetime(df['DateTime'])

# Calculate the average value for each day and month and convert the date column to a string
df_day = df.groupby(df['DateTime'].dt.date).mean().reset_index()
df_month = df.groupby(df['DateTime'].dt.to_period('M')).mean().reset_index().astype({'DateTime': 'str'})

# Write the summarized data to new tables in the same database
df_day.to_sql('air_2022_day', conn, if_exists='replace')
df_month.to_sql('air_2022_month', conn, if_exists='replace')

# Close the database connection
conn.close()